# Facebook Conversion API (CAPI) Tutorial

This notebook demonstrates how to:
1. **Fetch data from BigQuery** (containing `lead_id` and `phone`).
2. **Transform and Hash data** into Facebook's required format.
3. **Send events to Facebook Conversion API**.

---

## 1. Prerequisites

### Install Dependencies
Ensure you have the required libraries installed:
```bash
pip install google-cloud-bigquery pandas python-dotenv requests
```

### Setup Environment Variables
Create a `.env` file based on `.env.example` with your:
- `FB_ACCESS_TOKEN`: Your System User access token.
- `FB_PIXEL_ID`: The ID of the Pixel (Dataset) you are sending events to.
- `GOOGLE_APPLICATION_CREDENTIALS`: Path to your GCP Service Account JSON.
- `GCP_PROJECT_ID`: Your GCP Project ID.

In [ ]:
import os
import hashlib
import re
import pandas as pd
import requests
import time
from datetime import datetime
from google.cloud import bigquery
from dotenv import load_dotenv

# Load credentials
load_dotenv()

FB_ACCESS_TOKEN = os.getenv('FB_ACCESS_TOKEN')
FB_PIXEL_ID = os.getenv('FB_PIXEL_ID')
GCP_PROJECT_ID = os.getenv('GCP_PROJECT_ID')
API_VERSION = 'v24.0'

print(f"Setup complete. Using API Version: {API_VERSION}")

## 2. Fetch Data from BigQuery

We will fetch records that need to be sent as offline conversions. Usually, this includes a `lead_id` (from Facebook Lead Ads) and a `phone` number.

In [ ]:
def fetch_data_from_bq():
    client = bigquery.Client()
    
    # Example Query - Update with your actual table and logic
    query = f"""
        SELECT 
            lead_id, 
            phone, 
            event_name, 
            event_time, 
            value, 
            currency
        FROM `{GCP_PROJECT_ID}.your_dataset.your_table` 
        WHERE sent_to_fb IS FALSE
        LIMIT 10
    """
    
    # For demonstration, if you don't have BQ setup yet, we create a mock DataFrame
    try:
        # df = client.query(query).to_dataframe()
        # return df
        raise Exception("Skipping BQ for mock data demonstration")
    except:
        print("Using Mock Data for demonstration")
        mock_data = {
            'lead_id': ['123456789', '987654321'],
            'phone': ['+66-81-234-5678', '66 (89) 999 9999'],
            'event_name': ['Purchase', 'Purchase'],
            'event_time': [int(time.time()), int(time.time())],
            'value': [1500.0, 2500.0],
            'currency': ['THB', 'THB']
        }
        return pd.DataFrame(mock_data)

df = fetch_data_from_bq()
df.head()

## 3. Data Transformation & Hashing

Facebook requires PII (Personally Identifiable Information) like **Phone Numbers** to be hashed using **SHA256**. 

**Rules for Phone Numbers:**
1. Remove symbols, spaces, and leading zeros.
2. Include country code (e.g., `66` for Thailand).
3. Hash the resulting string.

In [ ]:
def hash_data(value):
    if pd.isna(value) or value == "":
        return None
    return hashlib.sha256(str(value).strip().lower().encode('utf-8')).hexdigest()

def prepare_fb_payload(row):
    # Normalize and hash phone
    # Remove all symbols and non-numeric characters using regex
    clean_phone = re.sub(r'\D', '', str(row['phone']))
    hashed_phone = hash_data(clean_phone)
    
    payload = {
        "event_name": row['event_name'],
        "event_time": int(row['event_time']),
        "action_source": "system_generated", # or "physical_store" for offline
        "user_data": {
            "ph": [hashed_phone],
            "lead_id": row['lead_id'] # lead_id does NOT need hashing
        },
        "custom_data": {
            "currency": row['currency'],
            "value": float(row['value'])
        }
    }
    return payload

# Apply transformation
df['fb_payload'] = df.apply(prepare_fb_payload, axis=1)
print("Sample Payload:")
print(df['fb_payload'].iloc[0])

## 4. Send to Facebook Conversion API

We use a `POST` request to the `/events` endpoint.

In [ ]:
def send_to_facebook(payload_list):
    url = f"https://graph.facebook.com/{API_VERSION}/{FB_PIXEL_ID}/events"
    params = {
        "access_token": FB_ACCESS_TOKEN
    }
    data = {
        "data": payload_list
    }
    
    response = requests.post(url, params=params, json=data)
    return response.json()

# Batch sending (Facebook allows up to 1000 events per request)
all_payloads = df['fb_payload'].tolist()

if not FB_ACCESS_TOKEN or "your_" in FB_ACCESS_TOKEN:
    print("⚠️ Please set your FB_ACCESS_TOKEN in .env to actually send data.")
else:
    result = send_to_facebook(all_payloads)
    print("Facebook API Response:")
    print(result)

## Summary
- **BigQuery**: Source of truth for your conversion events.
- **Hashing**: Critical step for privacy and matching. Only hash PII (Email, Phone, Name, etc.).
- **Lead ID**: Use `lead_id` under `user_data` to link conversions back to a specific lead form submission.
- **API Version**: Always check for the latest Graph API version.